In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="IMSyPP/hate_speech_en")

In [6]:
pipe("I hate you")

[{'label': 'LABEL_2', 'score': 0.6886570453643799}]

In [4]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
# SKLearn libraries
import tensorflow as tf
import tensorflow_io as tfio
import warnings
warnings.filterwarnings('ignore')

2024-03-04 10:55:34.424702: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 10:55:34.457658: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 10:55:34.457691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 10:55:34.458902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 10:55:34.464696: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 10:55:34.465745: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
# !curl -sSOL https://downloads.apache.org/kafka/3.3.2/kafka_2.13-3.3.2.tgz
# !tar -xzf kafka_2.13-3.3.2.tgz-

!./kafka/bin/zookeeper-server-start.sh -daemon ./kafka/config/zookeeper.properties 
!./kafka/bin/kafka-server-start.sh -daemon
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [1]:
# !./kafka/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic cancer-train

In [2]:
def error_callback(exc):
      raise Exception('Error while sending data to kafka: {0}'.format(str(exc)))


def write_to_kafka(topic_name, items):
      count=0
      producer = KafkaProducer(bootstrap_servers=['127.0.0.1:29092'])
      for message, key in items:
        print(message.encode('utf-8'))
        producer.send(topic_name,
                      key=key.encode('utf-8'),
                      value=message.encode('utf-8')).add_errback(error_callback)
        count+=1
      producer.flush()
      print("Wrote {0} messages into topic: {1}".format(count, topic_name))


def decode_kafka_item(message):
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
    return message.value.decode('utf-8')

      


In [3]:
import pytchat

chat = pytchat.create(video_id="uIx8l2xlYVY")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        write_to_kafka('youtube', [(c.message, c.author.name)])


2020-09-29 17:59:21 [qvioa]- gi


NameError: name 'KafkaProducer' is not defined

In [5]:
from kafka import KafkaConsumer

def read_from_kafka(topic_name):
    consumer = KafkaConsumer(
        topic_name,
        auto_offset_reset='earliest',
        bootstrap_servers=['127.0.0.1:29092'],
        consumer_timeout_ms=10000
    )
    # Continuously listen for messages
    while True:
        records = consumer.poll(timeout_ms=1000, max_records=500)
        print("Received {0} messages from topic: {1}".format(len(records), topic_name))
        for record in records:
            for message in records[record]:
                decoded_msg = decode_kafka_item(message)
                print(decoded_msg)

read_from_kafka('youtube')

Received 1 messages from topic: youtube
youtube:0:0: key=b'qvioa' value=b'gi'
gi
youtube:0:1: key=b'Marko Kro\xc5\xa1elj' value=b'hi'
hi
youtube:0:2: key=b'elia celegato' value=b'hi'
hi
youtube:0:3: key=b'Wagner neres' value=b'bora'
bora
youtube:0:4: key=b'pierenrico pranteddu' value=b'Hi ciao'
Hi ciao
youtube:0:5: key=b'Jonas' value=b'hi qvio'
hi qvio
youtube:0:6: key=b'Bob Ski' value=b'hey'
hey
youtube:0:7: key=b'Jes\xc3\xbas Galeas' value=b'HOLA'
HOLA
youtube:0:8: key=b'MAGNETO' value=b'Nasa + SpaceX = NasaX'
Nasa + SpaceX = NasaX
youtube:0:9: key=b'Lidia Torres' value=b'oi'
oi
youtube:0:10: key=b'Bluerizon' value=b'pablo Escobar is a Life!'
pablo Escobar is a Life!
youtube:0:11: key=b'Burak KARA' value=b'hi mydexpay'
hi mydexpay
youtube:0:12: key=b'ClearSky' value=b'Salut'
Salut
youtube:0:13: key=b'Mitra Saghri' value=b'hiiiiiii'
hiiiiiii
youtube:0:14: key=b'Fr3Svi' value=b'ciaoooooo'
ciaoooooo
youtube:0:15: key=b'Miho' value=b'hi'
hi
youtube:0:16: key=b'1224ChrisNg' value=b'hi'
hi

KeyboardInterrupt: 